# Part 3: Reacting to events

If a friend has high blood pressure we better let them know!

We'll need some stuff prom those previous notebooks...

In [ ]:
#Hour and minute when this started to create unique names
%store -r HM
#Home directory of the user running this notebook
%store -r home
#Private key file name, for AWS IoT authentication
%store -r private_key
#Certificate file name, for AWS IoT authentication
%store -r certificate_file
#Certificate authority file name, for AWS IoT authentication
%store -r ca_file
#Address to connect using MQTT
%store -r endpoint_address
#The name of the topic to send MQTT messages to
%store -r topic_name
#A list of sample messages
%store -r payloads
#A sample message payload
%store -r payload

Let's start with a very simple function that logs a simple message.


This is your code and could do anything: send an em-mail, a tweet, a phone call...

In [ ]:
# %load hypertension.py
def on_hbp(event, context):
    name = event['name']
    print("High blood pressure detected. Take care {}!".format(name))


To create a lambda function we need to:

- Create a zip package with the code

In [ ]:
zip_file = "{}/aws_iot/hypertension{}.zip".format(home,HM) 
!zip -g {zip_file} hypertension.py
zip_file

- Create a role authorizing the function execution

In [ ]:
print(open('lambda-trust.json').read())

In [ ]:
role_name = "fn_hbp_role{}".format(HM) 
fn_role_arn = !aws iam create-role \
    --role-name {role_name} \
    --assume-role-policy-document file://lambda-trust.json \
    --query Role.Arn \
    --output text
fn_role_arn = fn_role_arn.s
fn_role_arn

In [ ]:
policy_arn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole"
!aws iam attach-role-policy \
    --role-name '{role_name}' \
    --policy-arn '{policy_arn}'

In [ ]:
!echo {fn_role_arn}

Create the function:

In [ ]:
function_name="on_hbp{}".format(HM)
function_name

In [ ]:
function_arn = !aws lambda create-function \
    --function-name '{function_name}' \
    --zip-file "fileb://{zip_file}" \
    --role '{fn_role_arn}' \
    --handler hypertension.on_hbp \
    --runtime python3.6 \
    --timeout 30 \
    --memory-size 128 \
    --query FunctionArn \
    --output text
function_arn = function_arn.s
function_arn

Now let's create the rule that tells AWS IoT to call that function when a message with high blood pressure arrives.

For that we need:
- The SQL statement capturing messages with the desired condition

In [ ]:
sql = """
SELECT name, 
    time_stamp, 
    systolic, 
    diastolic, 
    id, 
    timestamp() as recv_stamp 
FROM '{}'
WHERE systolic > 120 
    OR diastolic > 80
""".format(topic_name)
print(sql)

- Authorize the rule to invoke the lambda function

https://docs.aws.amazon.com/iot/latest/developerguide/lambda-rule.html

In [ ]:
import json
rule = {
        "sql": sql, 
        "ruleDisabled": False,
        "awsIotSqlVersion": "2016-03-23",
        "actions": [{
            "lambda": {
                "functionArn": function_arn
             }
        }]}
rule = json.dumps(rule)
rule_file="{}/rule.json".format(home)
with open(rule_file, "w") as f:
    f.write(rule)
rule_file

In [ ]:
print(open(rule_file).read())

In [ ]:
rule_name="bp2lambda_rule{}".format(HM)
!aws iot create-topic-rule \
  --rule-name '{rule_name}' \
  --topic-rule-payload file://{rule_file}
rule_name

In [ ]:
rule_arn=!aws iot get-topic-rule --rule-name '{rule_name}' --query ruleArn --output text
rule_arn = rule_arn.s
rule_arn

In [ ]:
account_id=!aws sts get-caller-identity --query Account --output text
account_id =account_id.s
account_id

In [ ]:
unique_id = "unique{}".format(HM)
unique_id

In [ ]:
!aws lambda add-permission \
    --function-name "{function_name}" \
    --principal iot.amazonaws.com \
    --source-arn '{rule_arn}' \
    --source-account "{account_id}" \
    --statement-id "{unique_id}" \
    --action "lambda:InvokeFunction"

In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id="NotebookCient{}".format(HM) 
mqtt = AWSIoTMQTTClient(client_id)
mqtt.configureEndpoint(endpoint_address, 8883)
mqtt.configureCredentials(ca_file, private_key, certificate_file)
mqtt.configureConnectDisconnectTimeout(10)
mqtt.connect()
mqtt

In [ ]:
import random
payload = random.choice(payloads)
print(payload)
mqtt.publish(topic_name, payload , QoS = 0)

In [ ]:
log_group="/aws/lambda/{}".format(function_name)
print(log_group)
!aws logs describe-log-groups --log-group-name-prefix '{log_group}'

In [ ]:
log_stream = !aws logs describe-log-streams \
    --log-group-name '{log_group}' \
    --order-by LastEventTime \
    --max-items 1  \
    --query 'logStreams[0].logStreamName'\
    --output text
log_stream = log_stream.s
log_stream

In [ ]:
!aws logs get-log-events \
    --log-group-name '{log_group}' \
    --log-stream-name '{log_stream}' \
    --query 'events[].message' \
    --output text 

In [ ]:
mqtt.disconnect()

Now let's have some fun accessing real things.Let's see how we can use certificates to allow access from other AWS IoT Core Accounts. [Click here to start controling other things!](aws-iot-other-things.ipynb)